In [101]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
import cProfile
import json
import random

puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv'
my_submission_path = 'submission.csv'

puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)
my_submission_df = pd.read_csv(my_submission_path)

In [102]:
puzzles_df[puzzles_df["puzzle_type"] == 'globe_1/8']
# Two types: permutation & colors

,id,puzzle_type,solution_state,initial_state,num_wildcards
338,338,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,I;P;O;A;A;D;F;L;J;M;G;M;P;F;E;J;E;B;O;G;H;D;N;...,0
339,339,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,K;I;C;B;M;J;D;G;E;H;N;E;F;I;D;K;L;P;A;O;B;N;J;...,0
340,340,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,M;O;L;M;P;F;L;B;I;G;E;J;A;G;O;I;C;H;D;P;B;A;N;...,0
341,341,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,P;O;G;J;A;L;F;C;D;B;P;N;F;M;I;G;O;M;H;H;E;N;K;...,0
342,342,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,F;H;O;M;C;J;K;I;J;L;B;N;B;G;K;H;D;P;A;D;E;N;P;...,2
343,343,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N21;N1;N9;N20;N30;N8;N25;N26;N11;N2;N16;N27;N4...,0
344,344,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N30;N0;N22;N15;N24;N19;N14;N18;N31;N6;N23;N16;...,0
345,345,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N19;N11;N22;N29;N15;N7;N14;N16;N0;N18;N9;N24;N...,0
346,346,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N9;N7;N17;N11;N30;N14;N5;N0;N16;N31;N6;N4;N1;N...,0
347,347,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N26;N10;N17;N24;N11;N3;N14;N16;N23;N2;N21;N1;N...,0


# Solving 2xM

In [103]:
idx = 343
N = 1+1
M = int(2*8)

In [104]:
def print_globe(b):
    print(np.asarray(b).reshape(2, M))

apply_move = {}

def set_moves(verbose = False): # Assumes M is reset
    test_globe = np.array([i for i in range(2*M)])
    r0_perm = np.array([i for i in range(2*M)])
    r1_perm = np.array([i for i in range(2*M)])
    f0_perm = np.array([i for i in range(2*M)])
    
    r0_perm[M-1] = 0
    for i in range(0, M-1):
        r0_perm[i] = i+1
    
    r1_perm[2 * M - 1] = M
    for i in range(M, 2*M-1):
        r1_perm[i] = i+1
    
    for i in range(M//2):
        f0_perm[i] = M + M//2 - 1 - i
    
    for i in range(M, M + M//2):
        f0_perm[i] = M//2-1 - (i-M)
    
    nr0_perm = np.array([i for i in range(2*M)])
    nr1_perm = np.array([i for i in range(2*M)])
    
    for i in range(M-1):
        nr0_perm = nr0_perm[r0_perm]
    
    for i in range(M-1):
        nr1_perm = nr1_perm[r1_perm]

    def r0(arr, moves):
        moves.append('r0')
        arr = arr[r0_perm]
        return arr
    
    def nr0(arr, moves):
        moves.append('-r0')
        arr = arr[nr0_perm]
        return arr
    
    def r1(arr, moves):
        moves.append('r1')
        arr = arr[r1_perm]
        return arr
    
    def nr1(arr, moves):
        moves.append('-r1')
        arr = arr[nr1_perm]
        return arr
    
    def f0(arr, moves):
        moves.append('f0')
        arr = arr[f0_perm]
        return arr
    
    apply_move['r0'] = r0
    apply_move['-r0'] = nr0
    apply_move['r1'] = r1
    apply_move['-r1'] = nr1
    apply_move['f0'] = f0

    def create_f_function(i, cur_perm):
        return lambda arr, moves: (moves.append('f' + str(i)), arr[cur_perm])[1]

    for i in range(1, M):
        cur_perm = np.array([i for i in range(2*M)])
    
        for j in range(i):
            cur_perm = cur_perm[r0_perm]
            cur_perm = cur_perm[r1_perm]
    
        cur_perm = cur_perm[f0_perm]
    
        for j in range(i):
            cur_perm = cur_perm[nr0_perm]
            cur_perm = cur_perm[nr1_perm]
            
        apply_move['f' + str(i)] = create_f_function(i, cur_perm)

    if verbose:
        print("r0")
        print_globe(r0(test_globe, []))
        print("nr0")
        print_globe(nr0(test_globe, []))
        print("r1")
        print_globe(r1(test_globe, []))
        print("f0")
        print_globe(f0(test_globe, []))
        print_globe(apply_move['f1'](test_globe, []))
        print_globe(apply_move['f2'](test_globe, []))

    set_half_turn_moves(verbose)

def set_half_turn_moves(verbose):
    test_globe = np.array([i for i in range(2*M)])
    def half_turn_ccw(arr, moves):
        for i in range(M//2):
            arr = apply_move['-r0'](arr, moves)
        
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        
        for i in range(M//2):
            arr = apply_move['r0'](arr, moves)
        return arr
    
    def half_turn_cw(arr, moves):
        for i in range(M//2):
            arr = apply_move['-r0'](arr, moves)
        
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        
        for i in range(M//2):
            arr = apply_move['r0'](arr, moves)
        return arr
    
    apply_move['half_turn'] = half_turn_ccw
    apply_move['rev_half_turn'] = half_turn_cw

    def top_right_cw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    def top_right_ccw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    def top_left_ccw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r1'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    def top_left_cw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r1'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    apply_move['top_right_cw'] = top_right_cw
    apply_move['top_right_ccw'] = top_right_ccw
    apply_move['top_left_cw'] = top_left_cw
    apply_move['top_left_ccw'] = top_left_ccw

    if verbose:
        print_globe(half_turn_ccw(test_globe, []))
        print_globe(half_turn_cw(test_globe, []))
        print_globe(top_right_cw(test_globe, []))
        print_globe(top_right_ccw(test_globe, []))
        print_globe(top_left_cw(test_globe, []))
        print_globe(top_left_ccw(test_globe, []))

set_moves(True)

r0
[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15  0]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
nr0
[[15  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
r1
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 16]]
f0
[[23 22 21 20 19 18 17 16  8  9 10 11 12 13 14 15]
 [ 7  6  5  4  3  2  1  0 24 25 26 27 28 29 30 31]]
[[ 0 24 23 22 21 20 19 18 17  9 10 11 12 13 14 15]
 [16  8  7  6  5  4  3  2  1 25 26 27 28 29 30 31]]
[[ 0  1 25 24 23 22 21 20 19 18 10 11 12 13 14 15]
 [16 17  9  8  7  6  5  4  3  2 26 27 28 29 30 31]]
[[ 1  2  3  4  5  6  7 23  8  9 10 11 12 13 14 15]
 [ 0 16 17 18 19 20 21 22 24 25 26 27 28 29 30 31]]
[[16  0  1  2  3  4  5  6  8  9 10 11 12 13 14 15]
 [17 18 19 20 21 22 23  7 24 25 26 27 28 29 30 31]]
[[ 0  1  2  3  4  5  6  7 16  8  9 10 11 12 13 14]
 [17 18 19 20 21 22 23 15 24 25 26 27 28 29 30 31]]
[[ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 23]
 [ 8 16 1

## Phase 0: Set Left Half

In [105]:
def set_left_half(globe: np.array, solution_globe: np.array, moves: list) -> np.array:
    """
    MOVES CAN STILL BE OPTIMIZED. Optimize bottom left first.
    Note we can directly model some operations as a shortest path problem instead of constructing by hand.

    Sets the left half of the globe to anything (the right half is arbitrary after).
    Returns it (original unaffected).
    'moves' changed in place.

    We first set the top left then the bottom left, both from left to right.
    Most of the puzzles have duplicates, and this function accounts for that to decrease moves.

    TOP LEFT:
        For the top left, say we want to set the ith element.
        Then, our goal is to use any element with value ai (remember there are duplicates).
        Let's get it in the bottom row at position i+m/2-1 so we can apply fi to finish.

        Consider each candidate element and pick the one that takes the least moves.

        If it is in the top, try all available f's. Now it is in the bottom.

        Now the element must be in the bottom, say at position j. The following algo is called fastest_rotation() in the code:
        It will take j-(i+m/2-1) rotations (if negative, use inverse).
            POSSIBLE OPTIMIZATION: we could use 2 f moves to move the element faster. Use BFS for optimal.

        Finally, Apply fi to set the element.

    BOTTOM LEFT:
        For the bottom left, say we want to set the ith element.
            Note that i > 0, i=0 handled separately similar to TOP LEFT.
        An invariant is that the elements 0, 1, ... i-1 are contiguous and i-1 is at M+M//2-1 in the bottom (center).

        Consider each candidate element (any with value ai) and pick the one that takes the least moves.

        If it is in the top, use f to put it in the bottom, now it is in the right half.
        If it is in the bottom and in the left half, rotate until it is in the right half.

        Now the element must be in the right half. The following algo is called merge_i() in the code:
        Use r1 until element 0 is at index M or element i is at M + M//2, whichever occurs sooner.
        Then, use top_right_ccw until element i-1 is at M+M//2-1.
        Repeat until element i-1 is adjacent to element i (at M + M//2). Then finish with r1.

    NUMBER OF MOVES:
        Empirical results for 3/33:
            N0:N1... (no duplicates)
                Top left ~390
                Bot left ~1550

            (AAAAAABBBBBBC...) (6 original duplicates)
                Top left ~180
                Bot left ~800

        Theory:
            Let x be the number of duplicates.

            Top left:
                ~M/2(x+1) moves per element.
                    If x=1, the element is about M/4 away from i+m/2-1. We generalize this as x+1 points on a circle.
            Bottom left:
                ~2*M/(x+1)
                    Using the x+1 points on a circle argument, bottom point is M/(x+1) away.
                    To decrease dist by 1, we'll need r1 + 1 top top_right_ccw.
                    So worse case is 4 moves per 1 dist. However, for small i we can do multiple top_right_ccw at once.

    OPTIMAL PERM TO SET:
        WLOG consider setting the top left. It is optimal to set it to 0, 1, ... M/4-1, 3M/4, 2, ... M-1

        Move_top_bot_helper() will insert the remaining elements (M/4 ... 3M/4-1) randomly.

        Note that an array of length X has X^2/4 inversions on average.
        Thus, we'll have (M/2)^2/4 inversions from these remaining elements regardless of our choice of top left.

        Let's calculate the # of inversions between the remaining elements and those in the top left.
        It suffices to calculate it for a single element since they all have the same relative value with respect to top left.
        There are M//2+1 "locations" it could equally likely be (before 1st top left, between 1st, 2nd, ...).
        The inversions respectively are M/4, M/4-1, ... 0 ... M/4.
        Thus, the average # of inversions for each of remaining element with elements in top left = M/8.
        We can prove the optimality via:
            Inversions contributed by each element are independent.
            Minimum expected inversions achieved at median via calculus: minimize (k)(k-1)/2 + (m-k)(m-k-1)/2
        This yields M/8 * M/2 more inversions.

        The total is M^2/8 inversions, which is half the number if top was just randomly permuted (M^2/4).
    """

    # Optimal top left and bottom left
    sorted_top = list(solution_globe[:M])
    sorted_bot = list(solution_globe[M:])
    top_left = sorted_top[:M//4] + sorted_top[-(M//2 - M//4):]
    bot_left = sorted_bot[:M//4] + sorted_bot[-(M//2 - M//4):]

    def idx_after_f(x, i):
        # Index of i (i is in top) after using fx (gets mirrored and placed at bottom)
        return M + x + (M//2-1 - (i-x))

    def fastest_rotation(j, k):
        # Returns the shortest rotation moves to place j at k on bottom
        dist_r1 = (j-k) % M
        dist_nr1 = (k-j) % M
        if dist_r1 < dist_nr1:
            # Faster to use r1
            return ['r1'] * dist_r1
        else:
            # Faster to go in the other direction and use -r1
            return ['-r1'] * dist_nr1

    # Set top left element by element
    for i in range(M//2):
        if globe[i] == top_left[i]:
            # Lucky! It's already set.
            continue

        best_moves = None
        candidates = []  # (moves, index of candidate in bottom) after reducing to in bottom case

        # Top
        for j in range(i+1, M):
            if globe[j] == top_left[i]:
                # Try all possible f{k} moves
                for x in range(max(i+1, j-M//2+1), min(j, M//2)+1):
                    candidates.append(([f'f{x}'], idx_after_f(x, j)))

        # Bottom
        for j in range(M, 2*M):
            if globe[j] == top_left[i]:
                candidates.append(([], j))

        # Now we've reduces to bottom case
        # Find candidate which takes the least moves
        final_idx_bot = M + i+M//2-1  # Index we want the element before applying fi
        for cur_moves, bot_j in candidates:
            cur_moves += fastest_rotation(bot_j, final_idx_bot)

            # Update if better
            if best_moves is None or len(cur_moves) < len(best_moves):
                best_moves = cur_moves

        # Apply final fi
        best_moves.append(f'f{i}')

        # Apply moves to the globe
        for move in best_moves:
            globe = apply_move[move](globe, moves)


    # Set bottom left element by element

    def merge_i(idx_0, idx_i):
        # Returns the moves to make elements i-1 and i adjacent (i is in the bottom left)

        moves = []

        while idx_i - idx_0 > i:
            # Use r1 until idx_0 is at M or idx_i is at M + M//2
            max_r1 = min(idx_0 - M, idx_i - (M+M//2))
            moves.extend(['r1'] * max_r1)
            idx_i -= max_r1
            idx_0 -= max_r1

            # Use top_right_ccw until i-1 is at center (M + M//2-1)
            idx_i_minus_one = idx_0 + i-1
            amt_rotate = (M + M//2-1) - idx_i_minus_one
            moves.extend([f'f{M//2}'] + ['-r1']*amt_rotate + [f'f{M//2}'])
            idx_0 += amt_rotate

        return moves

    for i in range(M//2):
        best_moves = None

        if i == 0:
            # Handle element 0 separately, very similarly to how we did top left.
            candidates = []  # (moves, index of element) after reducing to in bottom case.

            # At top
            for j in range(M//2, M):
                if globe[j] == bot_left[i]:
                    candidates.append(([f'f{M//2}'], idx_after_f(M//2, j)))

            # At bot
            for j in range(M, 2*M):
                if globe[j] == bot_left[i]:
                    candidates.append(([], j))

            # Now we've reduced to bottom case
            # Find best candidate
            for cur_moves, idx_candidate in candidates:
                cur_moves += fastest_rotation(idx_candidate, M + M//2-1)

                # Update if better
                if best_moves is None or len(cur_moves) < len(best_moves):
                    best_moves = cur_moves
        else:
            candidates = [] # (moves, index of 0, index of candidate in bottom) after reducing to in bottom right case.
            idx_0 = M + M//2-i

            # Top
            for j in range(M//2, M):
                if globe[j] == bot_left[i]:
                    # Apply f{M//2}
                    candidates.append(([f'f{M//2}'], idx_0, idx_after_f(M//2, j)))

            # Bot left
            for j in range(M, idx_0):
                if globe[j] == bot_left[i]:
                    # Use r1 until candidate is in bot right
                    num_r1 = j-M+1
                    candidates.append((['r1'] * num_r1, idx_0 - num_r1, 2*M-1))

            # Bot right
            for j in range(M + M//2, 2*M):
                if globe[j] == bot_left[i]:
                    candidates.append(([], idx_0, j))

            # Now we've reduced to bottom right case
            # Find best candidate
            for cur_moves, idx_0, idx_candidate in candidates:
                cur_moves += merge_i(idx_0, idx_candidate)

                # Update if better
                if best_moves is None or len(cur_moves) < len(best_moves):
                    best_moves = cur_moves

            # Apply r1 to now have element i at center (M + M//2-1)
            best_moves.append('r1')

        # Apply moves to the globe
        for move in best_moves:
            globe = apply_move[move](globe, moves)

    assert (globe[:M//2] == top_left).all()
    assert (globe[M:M+M//2] == bot_left).all()

    return globe

## Phase 1: Sort Elements Into Top/Bottom Arrays

In [106]:
test_globe = np.array(puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][idx].split(';'))
test_solution_globe = np.array(puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['solution_state'][idx].split(';'))
print(len(test_globe))

32


In [107]:
# test_globe = apply_move['f0'](test_globe, [])
# test_globe = apply_move['f9'](test_globe, [])
# test_globe = apply_move['r0'](test_globe, [])
# test_globe = apply_move['r0'](test_globe, [])
# test_globe = apply_move['f1'](test_globe, [])

In [108]:
print_globe(test_globe)

[['N21' 'N1' 'N9' 'N20' 'N30' 'N8' 'N25' 'N26' 'N11' 'N2' 'N16' 'N27'
  'N4' 'N28' 'N6' 'N13']
 ['N17' 'N31' 'N12' 'N18' 'N19' 'N0' 'N29' 'N10' 'N22' 'N14' 'N24' 'N7'
  'N3' 'N5' 'N15' 'N23']]


In [109]:
moves = []

In [138]:
def is_moved_top_bottom(arr, solution_arr):
    top = set(solution_arr[:M])
    bot = set(solution_arr[M:])

    for i in range(M):
        if arr[i] in bot:
            return False
    for i in range(M, 2*M):
        if arr[i] in top:
            return False
    return True

# change this to allow all M cyclic rotates
def move_top_bottom_helper(arr, solution_arr, moves):
    """
    Note that top_right_cw is just a shifted top_left_cw, so that is redundant.
    Also, we should apply the _right_cw directly at top element to remove rotating to right place.
    """
    
    top = set(solution_arr[:M])
    bot = set(solution_arr[M:])
    
    while not is_moved_top_bottom(arr, solution_arr):
        
        rotate_locs = [
            (M//2, M + M // 2 - 1, 'top_right_ccw'),
            (M-1, M, 'top_right_cw'),
            (M//2-1, M + M//2, 'top_left_cw'),
            (0, 2*M-1, 'top_left_ccw')
        ]

        all_costs = []

        for a, b, op_name in rotate_locs:
            mn_a, mn_b = int(1e9), int(1e9)
            top_ops, bot_ops = [], []
            
            for i in range(M):
                if arr[i] in bot: # TODO: Try no abs. Can only r0
                    if abs(i - a) < mn_a:
                        mn_a = abs(i-a)

                        if i < a: # rotate right
                            top_ops = ['-r0'] * (a - i)
                        else:
                            top_ops = ['r0'] * (i - a)

            for i in range(M, 2*M):
                if arr[i] in top:
                    if abs(i - b) < mn_b:
                        mn_b = abs(i - b)

                        if i < b:
                            bot_ops = ['-r1'] * (b - i)
                        else:
                            bot_ops = ['r1'] * (i - b)

            all_costs.append((mn_a + mn_b, top_ops + bot_ops, op_name))

        all_costs = list(sorted(all_costs))
        random_i = random.randrange(len(all_costs))  # Random choice
        for move in all_costs[random_i][1]:
            arr = apply_move[move](arr, moves)
        arr = apply_move[all_costs[random_i][2]](arr, moves)
        
    return arr

m = []
print_globe(move_top_bottom_helper(test_globe, test_solution_globe, m))
print(len(m))

IndexError: index 32 is out of bounds for axis 0 with size 32

In [111]:
test_globe = move_top_bottom_helper(test_globe, test_solution_globe, moves)
top_bot_moves = moves
top_bot_state = test_globe
# try only applying ex-red here

## Phase 2: Remove Inversions

In [112]:
test_globe = top_bot_state
moves = top_bot_moves

print_globe(test_globe)

[['N13' 'N1' 'N9' 'N8' 'N11' 'N2' 'N4' 'N5' 'N6' 'N3' 'N7' 'N10' 'N14'
  'N0' 'N12' 'N15']
 ['N28' 'N21' 'N31' 'N18' 'N19' 'N29' 'N22' 'N24' 'N27' 'N23' 'N26' 'N16'
  'N25' 'N17' 'N30' 'N20']]


In [113]:
def merge_and_count(arr, temp, left, mid, right):
    i, j, k = left, mid + 1, left
    inv_count = 0

    while i <= mid and j <= right:
        if arr[i] <= arr[j]:
            temp[k] = arr[i]
            i += 1
        else:
            temp[k] = arr[j]
            inv_count += (mid - i + 1)
            j += 1
        k += 1

    while i <= mid:
        temp[k] = arr[i]
        k += 1
        i += 1

    while j <= right:
        temp[k] = arr[j]
        k += 1
        j += 1

    for i in range(left, right + 1):
        arr[i] = temp[i]

    return inv_count

def merge_sort_and_count(arr, temp, left, right):
    inv_count = 0
    if left < right:
        mid = (left + right) // 2
        inv_count += merge_sort_and_count(arr, temp, left, mid)
        inv_count += merge_sort_and_count(arr, temp, mid + 1, right)
        inv_count += merge_and_count(arr, temp, left, mid, right)
    return inv_count

def count_inversions(arr):
    n = len(arr)
    temp_arr = arr.copy()  # Create a copy of the array
    temp = [0] * n
    return merge_sort_and_count(temp_arr, temp, 0, n - 1)

print(count_inversions(test_globe[0:M]))
print(count_inversions(test_globe[M:2*M]))

69
66


In [114]:
temp_test_globe = np.array([i for i in range(2*M)])

# swaps i, i+1 for both top and bottom
def swap_same_idx(i, arr, moves):
    loc = M // 2 - 1
    if i < loc:
        i += M

    delta = i - loc

    arr = apply_move['f' + str(delta)](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f' + str(delta)](arr, moves)

    arr = apply_move['r1'](arr, moves)
    arr = apply_move['r1'](arr, moves)

    plus_half_minus_one = (delta + loc) % M

    arr = apply_move['f' + str(plus_half_minus_one)](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f' + str(plus_half_minus_one)](arr, moves)

    arr = apply_move['-r0'](arr, moves) # could remove, but should be handled by ex-red
    arr = apply_move['-r1'](arr, moves) # could remove, but should be handled by ex-red

    return arr

print_globe(temp_test_globe)
print_globe(swap_same_idx(1, temp_test_globe, []))

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
[[ 0  2  1  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 18 17 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [115]:
def get_inversion_locs(arr):
    A, B = [], []
    for i in range(0, M-1):
        if (arr[i] > arr[i+1]):
            A.append(i)

    for i in range(M, 2*M-1):
        if (arr[i] > arr[i+1]):
            B.append(i - M)

    return (A, B)

# Could try: Non-circular min_abs_diff
def min_abs_diff(A, B):
    return (A[0], B[0]) # CHANGE POLICY
    # A = A + [i + M for i in A]
    # B = B + [i + M for i in B]
    
    # i, j = 0, 0
    # min_diff = int(1e9)
    # min_indices = (-1, -1)

    # while i < len(A) and j < len(B):
    #     diff = abs(A[i] - B[j])

    #     if diff < min_diff:
    #         min_diff = diff
    #         min_indices = (A[i], B[j])

    #     if A[i] < B[j]:
    #         i += 1
    #     else:
    #         j += 1

    # if min_indices[0] >= M:
    #     min_indices = (min_indices[0] - M, min_indices[1])

    # if min_indices[1] >= M:
    #     min_indices = (min_indices[0], min_indices[1] - M)
        
    # return min_indices

def is_array_zero_to_n(arr):
    n = len(arr) - 1
    return len(arr) == n + 1 and set(arr) == set(range(n + 1))

def check_swaps(arr, swaps):
    arr_copy = arr.copy()
    for swap in swaps:
        if 0 <= swap < len(arr_copy) - 1:
            arr_copy[swap], arr_copy[swap + 1] = arr_copy[swap + 1], arr_copy[swap]
            # print(arr_copy)
        else:
            return False
    return arr_copy == sorted(arr_copy)

# Phase 2, finding optimal order of swaps of one array to minimize the amount of rotations of other
# Minimize sum of absolute distances between swaps
def get_swaps_order(arr):
    swaps = []
    arr = [i % M for i in arr]

    mx = -1
    move_forward = []
    for i in range(len(arr)):
        if(arr[i] < mx):
             move_forward.append(i)
        mx = max(mx, arr[i])

    while move_forward:
        for i in range(len(move_forward)-1, -1, -1):
            if i != 0 and move_forward[i-1] == move_forward[i] - 1:
                continue

            swaps.append(move_forward[i] - 1)
            arr[move_forward[i]-1], arr[move_forward[i]] = arr[move_forward[i]], arr[move_forward[i]-1]
            move_forward[i] -= 1
            
            temp = -1
            for k in range(move_forward[i]):
                temp = max(temp, arr[k])

            
            if temp <= arr[move_forward[i]]:
                move_forward.pop(i)
                
            break

    if (not is_array_zero_to_n(arr)):
        print("get_swaps_order function is incorrect")
        # swaps = swaps + get_swaps_order(arr)
    
    return swaps

x = get_swaps_order([2,6,0,5,4,7,1,3])

In [116]:
def is_array_zero_to_n(arr):
    n = len(arr) - 1
    for i in range(len(arr)):
        if arr[i] != i:
            return False

    return True

def remove_inversions(arr, moves):
    
    # Phase 1: Remove until equal inversions
    up_inversions, down_inversions = get_inversion_locs(arr)
    up_cnt, down_cnt = count_inversions(arr[0:M]), count_inversions(arr[M:2*M])
    
    while up_cnt != down_cnt:
        if up_cnt == 0 or down_cnt == 0:
            print("Remove inversions failure: one array has no inversions, other still has")
        
        if up_cnt > down_cnt:
            arr = swap_same_idx(up_inversions[0], arr, moves) # CHANGE POLICY
        else:
            arr = swap_same_idx(down_inversions[0], arr, moves) # CHANGE POLICY

        up_cnt, down_cnt = count_inversions(arr[0:M]), count_inversions(arr[M:2*M])
        up_inversions, down_inversions = get_inversion_locs(arr)
        
    up_cnt, down_cnt = count_inversions(arr[0:M]), count_inversions(arr[M:2*M])
    
    # Phase 2: Both Have Inversions
    while up_inversions and down_inversions:
        up_i, down_i = min_abs_diff(up_inversions, down_inversions)

        if down_i < up_i: 
            diff = up_i - down_i # Could do: Try swapping K/2 left up, K/2 down right
            for k in range(diff):
                arr = apply_move['r0'](arr, moves)

            arr = swap_same_idx(down_i, arr, moves)

            for k in range(diff):
                arr = apply_move['-r0'](arr, moves)

        else:
            diff = down_i - up_i # Could do: Try swapping K/2 left up, K/2 down right
            for k in range(diff):
                arr = apply_move['r1'](arr, moves)

            arr = swap_same_idx(up_i, arr, moves)

            for k in range(diff):
                arr = apply_move['-r1'](arr, moves)

        up_inversions, down_inversions = get_inversion_locs(arr)
        
    # Phase 3: Do some final rotations to get everything aligned right. Check correctness
    while arr[0] != 0:
        arr = apply_move['r0'](arr, moves)

    while arr[M] != M:
        arr = apply_move['r1'](arr, moves)

    return arr


In [117]:
def min_inversion_ordering_array(arr: np.array, sorted_arr) -> np.array:
    """
    globe is a 2xm globe.

    When globe has duplicates, there are many sorted "orderings" when treating each element as distinct.

    For example:
    AABBCC is the original array.
    Then there are 2^3 sorted orderings e.g. 012345, 102345, 103245 ...

    This function returns the ordering that has minimum inversions.
    The optimal ordering has elements of the same value mapped to the
    corresponding increasing subsequence. Proof via exchange argument.

    For example (one layer only):
    BACCAB => 204513
    """

    res = np.ones(M, dtype=np.int16)
    idx = {}  # idx[value] = next index for 'value'

    # Get the first index in the solution array
    for i, x in enumerate(sorted_arr):
        if x not in idx:
            idx[x] = i

    # Find the ordering
    for i in range(M):
        res[i] = idx[arr[i]]
        idx[arr[i]] += 1

    return res

In [118]:
def min_inversion_ordering(globe: np.array, solution_globe) -> np.array:
    """
    Returns the ordering of elements that minimizes inversion in the
    bottom and top layer. See min_inversion_ordering_array for more details.
    """

    return np.concatenate((min_inversion_ordering_array(globe[:M], solution_globe[:M]),
                           min_inversion_ordering_array(globe[M:], solution_globe[M:]) + M))

In [119]:
def sort_top_bot(arr, solution_arr):
    """
    For top and bottom, find the rotation with minimum inversions for each parity.
    
    Then solve the pair with minimum sum of inversions.
    Note we don't factor in the rotations necessary, but this is negligible 
        (maybe add a penalty = rotations/(moves to remove 1 inversion))
    """
    
    # best_rotations[parity][0 if top else 1] = (inversions, rotation moves)
    best_rotations = [[(1e9,), (1e9,)], [(1e9,), (1e9,)]]
    
    # Try all possible rotations
    for dir in ('', '-'):
        for i in range(M//2):
            # Top
            cur_moves = []
            cur_arr = arr
            for rotation in range(i):
                cur_arr = apply_move[dir + 'r0'](cur_arr, cur_moves)
            cur_arr = min_inversion_ordering(cur_arr, solution_arr)
            inversions = count_inversions(cur_arr[:M])
            
            if best_rotations[inversions % 2][0][0] > inversions:
                best_rotations[inversions % 2][0] = (inversions, cur_moves)
            
            # Bottom
            cur_moves = []
            cur_arr = arr
            for rotation in range(i):
                cur_arr = apply_move[dir + 'r1'](cur_arr, cur_moves)
            cur_arr = min_inversion_ordering(cur_arr, solution_arr)
            inversions = count_inversions(cur_arr[M:])

            if best_rotations[inversions % 2][1][0] > inversions:
                best_rotations[inversions % 2][1] = (inversions, cur_moves)
        
    # TODO: Apply Exp-Red
    best = 0
    
    best_inversions = 1e9
    for i in range(2):
        inversions = best_rotations[i][0][0] + best_rotations[i][1][0]
        
        if inversions < best_inversions:
            best = i
            best_inversions = inversions
    
    if best_inversions == 1e9:
        # Possible that there aren't 2 rotations with same parity if duplicates
        return -1
    
    print(f"{best_inversions=}")
    
    moves = []
    for move in best_rotations[best][0][1] + best_rotations[best][1][1]:
        arr = apply_move[move](arr, moves)
    arr = min_inversion_ordering(arr, solution_arr)
    remove_inversions(arr, moves)
    
    return moves

res = sort_top_bot(test_globe, test_solution_globe)

best_inversions=90


In [120]:
print(len(res))

917


In [121]:
test_globe_copy = test_globe
for i in res:
    test_globe_copy = apply_move[i](test_globe_copy, [])
print_globe(test_globe_copy)

moves = moves + res
print(len(moves))

test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][idx]
test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
print_globe(test_globe)

for i in moves:
    test_globe = apply_move[i](test_globe, [])
print_globe(test_globe)

[['N0' 'N1' 'N2' 'N3' 'N4' 'N5' 'N6' 'N7' 'N8' 'N9' 'N10' 'N11' 'N12'
  'N13' 'N14' 'N15']
 ['N16' 'N17' 'N18' 'N19' 'N20' 'N21' 'N22' 'N23' 'N24' 'N25' 'N26' 'N27'
  'N28' 'N29' 'N30' 'N31']]
961
[[21  1  9 20 30  8 25 26 11  2 16 27  4 28  6 13]
 [17 31 12 18 19  0 29 10 22 14 24  7  3  5 15 23]]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]


# Transition

In [122]:
def get_run_length_encoding(path):    
    rows = [[] for _ in range(2)]
    
    def expand_f(x):
        for i in range(2):
            rows[i].append(x)
            rows[i].append('f0')
            rows[i].append(M - x)
    
    for i in path:
        if 'f' in i:
            if '-' in i:
                expand_f(int(i[2:]))
            else:
                expand_f(int(i[1:]))
        else:
            for j in range(2):
                temp = 'r' + str(j)

                if temp == i:
                    rows[j].append(1)
                    
                temp = '-r' + str(j)

                if temp == i:
                    rows[j].append(M-1)

    Rrows = [[] for _ in range(2)]

    for r in range(2):
        run = 0
        for i in rows[r]:
            if type(i) == type(0):
                run += i
            else:
                Rrows[r].append(run)
                Rrows[r].append('f0')
                run = 0
            
        Rrows[r].append(run)
    
    return (Rrows[0], Rrows[1])

In [123]:
def check_moves(arr, moves):
    for i in moves:
        arr = apply_move[i](arr, [])
    return is_array_zero_to_n(arr)

def solve_2xM(arr, solution_arr):
    # Set left half
    left_half_moves = []
    # arr = set_left_half(arr, solution_arr, left_half_moves)

    while True:
        # Move top/bot
        move_top_bot_moves = []
        arr = move_top_bottom_helper(arr, solution_arr, move_top_bot_moves)

        # Sort top/bot
        sort_moves = sort_top_bot(arr, solution_arr)
        
        if sort_moves != -1:
            break
        
    # Combine moves
    res_moves = left_half_moves + move_top_bot_moves + sort_moves

    print("Left Half Moves", len(left_half_moves),
          "Move top/bot moves:", len(move_top_bot_moves),
          "Sorting moves:", len(sort_moves))
    
    # res_moves = ex_red(res_moves)

    # print("After Ex-Red", len(res_moves))
    return res_moves
    

def solve_1xM(arr, sol_arr):
    res = []
    while(arr != sol_arr):
        arr = arr[1:] + [arr[0]]
        res.append('r0')
    return res

# test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][343]
# test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
# print_globe(test_globe)
# temp = solve_2xM(test_globe)

# Solving N x M

In [124]:
# GLobe 1/16
# idx = 348
# N = 1+1
# M = int(2*16)

In [130]:
# #Globe 3/33
# idx = 395
# N = 3+1 
# M = int(2*33)

In [131]:
# GLobe 8 / 25
idx = 396
N = 8+1 
M = int(2*25)

In [132]:
puzzles_df.iloc[idx]

id                                                              396
puzzle_type                                              globe_8/25
solution_state    A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...
initial_state     V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...
num_wildcards                                                     0
Name: 396, dtype: object

In [133]:
# Make all encodings same length
def equalize_lengths(encodings):
    mx_length = 0
    for i in encodings:
        mx_length = max(mx_length, len(i))

    for i in range(len(encodings)):
        if(((mx_length - len(encodings[i]))/2) % 2 == 1):
            print("Failure: Odd number of F0s needed to allign rows!")

        while(len(encodings[i]) < mx_length):
            encodings[i].append('f0')
            encodings[i].append(0)

    return encodings

def is_int(x):
    return isinstance(x, int)

def combine_palindromic_rows(encodings):
    encodings = equalize_lengths(encodings)
    n, m = len(encodings), len(encodings[0]) # number of rows / length of run-length encoding of each row

    # Take modulo + Convert F0s to FXs
    for j in range(m):
        all_prev = []
        for i in range(n):
            if is_int(encodings[i][j]): # Is rotation
                encodings[i][j] %= M
            else:
                all_prev.append(encodings[i][j-1])

        all_prev = list(sorted(all_prev))
        med = (len(all_prev) == 0) or all_prev[n//2]
        
        for i in range(n):  
            if not is_int(encodings[i][j]): # Is f0
                encodings[i][j-1] = (encodings[i][j-1] - med + M) % M
                encodings[i][j] = 'f' + str(med)
                encodings[i][j+1] += med

    # Convert to inverse rotation when better
    moves = []
    for j in range(m):
        if not is_int(encodings[0][j]):
            moves.append(encodings[0][j])
            continue

        for i in range(n):
            if M - encodings[i][j] < encodings[i][j]:
                moves = moves + ['-r' + str(i)] * (M - encodings[i][j])
            else:
                moves = moves + ['r' + str(i)] * (encodings[i][j])

    # print(moves)
    # Remove duplicate FXs
    final_moves = []
    i = 0
    while i < len(moves):
        if i != len(moves) - 1 and ('f' in moves[i]) and moves[i] == moves[i+1]:
            i += 2
        else:
            final_moves.append(moves[i])
            i += 1
    
    return final_moves

In [134]:
# redefine moves

initial_state = puzzles_df['initial_state'][idx].split(';')
solution_state = puzzles_df['solution_state'][idx].split(';')

all_encodings = []
top_half_moves = []
bot_half_moves = []

for i in range(N):
    if N-1-i < i:
        break
        
    arr = np.array(initial_state[i*M : i*M + M] 
                   + initial_state[(N-1-i) * M : (N-1-i) * M + M])
    solution_arr = np.array(solution_state[i*M : i*M + M] 
                            + solution_state[(N-1-i) * M : (N-1-i) * M + M])

    set_moves()

    if N-1-i == i:
        # Set moves for 1xM separately
        # arr = np.array(initial_state[i*M : i*M + M])
        
        cur_moves = solve_1xM(initial_state[i*M : i*M + M], solution_state[i*M : i*M + M])
        top_half_moves.append(get_run_length_encoding(cur_moves)[0])

        continue
    
    cur_moves = solve_2xM(arr, solution_arr) 

    # Try red-expand for palindromic rows separately first
    cur_top_moves, cur_bot_moves = get_run_length_encoding(cur_moves)
    cur_moves = combine_palindromic_rows([cur_top_moves] + [cur_bot_moves])
    # End
    
    cur_top_moves, cur_bot_moves = get_run_length_encoding(cur_moves)

    top_half_moves.append(cur_top_moves)
    bot_half_moves.append(cur_bot_moves)
    
all_encodings = top_half_moves + bot_half_moves[::-1]

best_inversions=942
Left Half Moves 0 Move top/bot moves: 131 Sorting moves: 12104
best_inversions=984
Left Half Moves 0 Move top/bot moves: 130 Sorting moves: 11043
best_inversions=928
Left Half Moves 0 Move top/bot moves: 144 Sorting moves: 12758
best_inversions=916
Left Half Moves 0 Move top/bot moves: 129 Sorting moves: 12952


In [135]:
print(len(all_encodings))

9


In [136]:
res_moves = combine_palindromic_rows(all_encodings)

In [137]:
print(len(res_moves))

41522


In [467]:
print(res_moves)

['f0', 'r0', 'r3', 'r3', 'f65', '-r2', '-r3', 'f65', 'r0', 'r0', '-r2', '-r2', 'f65', 'r0', '-r2', 'f65', 'r0', '-r2', 'f65', 'r0', 'r0', 'r1', 'r1', 'r1', 'r2', 'f64', '-r2', '-r2', '-r3', 'f64', 'r1', '-r3', 'f64', 'r1', 'r1', '-r3', 'f64', 'r0', 'r1', 'r1', 'r1', 'f63', 'r1', '-r3', 'f63', 'r0', 'r0', 'r0', '-r2', 'f63', '-r2', '-r3', 'f64', 'r1', '-r3', '-r3', '-r3', 'f64', '-r2', '-r3', 'f64', 'r0', 'r0', 'r1', 'f64', 'r0', '-r2', '-r3', 'f64', 'r0', '-r2', '-r2', '-r3', 'f64', '-r2', '-r2', '-r3', 'f65', 'r0', 'r1', 'f65', 'r1', 'r1', 'r1', 'r1', 'r1', '-r3', 'f65', 'r1', 'r1', '-r3', 'f65', 'r0', 'r0', 'r0', 'r0', 'r1', 'r1', 'r1', 'r1', 'r3', 'r3', 'r3', 'f62', 'r0', '-r2', '-r2', '-r2', '-r3', 'f62', 'r0', '-r2', 'f62', 'r1', 'r1', 'r1', '-r3', '-r3', 'f62', '-r2', '-r3', 'f62', 'r0', '-r2', '-r3', 'f62', '-r2', '-r3', 'f62', 'r0', '-r2', 'f62', '-r2', '-r2', '-r3', 'f63', '-r2', '-r2', '-r3', '-r3', 'f63', '-r2', '-r2', '-r3', '-r3', 'f64', 'r0', '-r2', '-r3', 'f64', '-r2', '

# Checking Moves With Official

In [457]:
def getInversePerm(arr):
    # gets the inverse move for a certain move
    res = [0 for i in range(len(arr))]
    for i in range(len(arr)):
        res[arr[i]] = i
    return res
    
# type : (np.array(move_perm_i), np.array(name_i))
puz_info = {}

# type : {move : perm}
move_to_perm = {}

puzzle_info_df['allowed_moves_converted'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

for i in range(len(puzzle_info_df)):
    puz_info[puzzle_info_df['puzzle_type'][i]] = [[], []]
    move_to_perm[puzzle_info_df['puzzle_type'][i]] = {}
    
    for j in puzzle_info_df['allowed_moves_converted'][i].keys():
        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(j)
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(puzzle_info_df['allowed_moves_converted'][i][j]))

        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(str('-' + j)) # might be the opposite
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(getInversePerm(puzzle_info_df['allowed_moves_converted'][i][j])))

        move_to_perm[puzzle_info_df['puzzle_type'][i]][str('-' + j)] = np.array(getInversePerm(puzzle_info_df['allowed_moves_converted'][i][j]))
        move_to_perm[puzzle_info_df['puzzle_type'][i]][j] = np.array(puzzle_info_df['allowed_moves_converted'][i][j])

# move_to_perm['cube_2/2/2']

In [458]:
def check_official(moves, idx):
    puzzle_type = puzzles_df['puzzle_type'][idx]
    initial_state = np.array(puzzles_df['initial_state'][idx].split(';'))
    solution_state = np.array(puzzles_df['solution_state'][idx].split(';'))

    official_moves = move_to_perm[puzzle_type]
    
    for i in moves:
        initial_state = initial_state[official_moves[i]]

    return all(solution_state == initial_state)

check_official(res_moves, idx)

True

# Submissions #

The submission format requires moves in a sequence to be delimited by a period `.`. Indicate the inverse of a named move with a preceeding `-`. Moves are applied from left to right.

The `sample_submission.csv` contains a baseline solution for each puzzle.

Let's check that the given sequence for the first puzzle is actually a solution.